# Goal : Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given video.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


### 1. Input Data 
 - from github 

In [ ]:
%cd /content
!mkdir input
!wget https://github.com/jeeenn/SportsFriends/raw/master/Data/yogaFriend_demo.mp4 -P /content/input
!wget https://github.com/jeeenn/SportsFriends/raw/master/Data/yogaFriend_demo_std.mp4 -P /content/input

## 2. Install OpenPose

In [2]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz

In [ ]:
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev


In [ ]:
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

## 3. Detect poses on a test video

We are going to detect poses on the following yoga video:

*   set input/output/model path 
*   set input file type/input_cut_time

In [5]:
import glob
import os
from google.colab import files

model_path = '/content/openpose'  
#input_path = '/content/drive/My Drive/yoga_video'
input_path = '/content/input'
output_path = '/content/output'
output_file_path = "".join([output_path, '/file'])
output_json_path = "".join([output_path, '/json'])

os.system(f"rm -rf '{output_path}'")
os.mkdir(output_path)
os.mkdir(output_file_path)
os.mkdir(output_json_path)

input_file_type = "mp4"
input_cut_time = 10

%cd $model_path

/content/openpose


*   *file cut /detecte key point /output : video file & json*
*   origin_file_path = 'content/output/yogaFriend_demo.mp4' <br/>
    origin_file_name = 'yogaFriend_demo.mp4' <br/>
    temp_file_name = 'content/output/openpose.avi' <br/>
*   Output <br/>
    output_file_name = 'content/output/file/yogaFriend_demo.mp4' <br/>
    output_json_name = 'content/output/json/yogaFriend_demo_xxx.json'

In [6]:
def detect_keypoint_file(file_name):
  
  origin_file_path = "".join([output_path, '/', file_name.split('/')[-1]])
  origin_file_name = file_name.split('/')[-1].split('.')[0]  
  temp_file_name = "".join([output_path, '/', 'openpose.avi'])
  output_file_name = "".join([output_file_path, '/', origin_file_name, '.', input_file_type])
  output_json_name = "".join([output_json_path, '/', origin_file_name])
    
  os.system(f"ffmpeg -y -loglevel info -i '{file_name}' -t {input_cut_time} '{origin_file_path}'")
  os.system(f"./build/examples/openpose/openpose.bin --keypoint_scale 3 --frame_step 3 --video '{origin_file_path}' --write_json '{output_json_name}/' --display 0 --write_video '{temp_file_name}'")
  os.system(f"ffmpeg -y -loglevel info -i '{temp_file_name}' '{output_file_name}'")
  os.system(f"rm -rf '{origin_file_path}'")
  os.system(f"rm -rf '{temp_file_name}'")
  

In [9]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

* file(video).zip / json.zip 생성
* local machine download

In [7]:
def download_file():
  file_zip_name = "".join([output_file_path, '.zip'])
  json_zip_name = "".join([output_json_path, '.zip'])
  zip_name = "".join([output_path, '.zip'])

  #os.system(f"zip -s 25 -r {file_zip_name} {output_file_path}")
  #files.download(file_zip_name)
  
  #os.system(f"zip -s 25 -r {json_zip_name} {output_json_path}")
  #files.download(json_zip_name)  

  os.system(f"zip -s 25 -r {zip_name} {output_path}")
  files.download(zip_name)  

*   loop input file count

In [10]:
for file_name in glob.glob("".join([input_path, '/*', input_file_type])):     
  detect_keypoint_file(file_name)  

In [11]:
show_local_mp4_video('/content/output/file/yogaFriend_demo.mp4', width=480, height=360) 

In [12]:
show_local_mp4_video('/content/output/file/yogaFriend_demo_std.mp4', width=480, height=360)

## 4. Compute poses 
  - base : key points ( json )
  - PDJ

In [ ]:
%cd /content/output
!rm -f compute_pdj.py
!rm -f compute_pdj.txt

In [14]:
%%writefile compute_pdj.py
from os import listdir
from os.path import isfile, join
import json
from scipy.spatial import distance
import math

# json파일 폴더에서 n개의 json파일을 읽어 (x, y) tuple이 들어있는 4500 (25 keypoints * n frame) 벡터 추출
def read_vec(json_path):
    json_path = json_path
    json_files = [f for f in listdir(json_path) if isfile(join(json_path, f))]
    json_files.sort()

    keypoint_data = []
    for json_file in json_files:
        #print(json_file)
        with open(json_path+json_file) as f:
            data = json.load(f)
        ary = data["people"][0]["pose_keypoints_2d"]
        del ary[2::3]

        # 코의 좌표 추출
        nose_x = ary[0]
        nose_y = ary[1]

        odd = ary[0::2]
        even = ary[1::2]

        # 코의 좌표가 (0, 0)이 되게끔 모든 keypoint 위치를 이동 (shift)
        origin_x = [x - nose_x for x in odd]
        origin_y = [y - nose_y for y in even]

        # 각 keypoint의 위치 정보를 (x, y)로 묶어서 list 안에 넣음
        keypoint_data += tuple(zip(origin_x, origin_y))

    return keypoint_data

# PDJ (Percentage of Detected Joints) 계산식 중 diagonal에 해당하는 값을 base (length between nose and middle hip)의 형태로 구함
def get_base(v1, v2):
    # 스승과 학생의 base 길이를 각각 구함
    base1 = distance.euclidean(v1[0], v1[8]) # 스승
    base2 = distance.euclidean(v2[0], v2[8]) # 학생

    scaling_factor = base1 / base2
    
    # 두 base의 평균을 기준 길이(PDJ의 diagonal)로 정함
    result = (base1 + base2) / 2
    return scaling_factor, result

# OKS (Object Keypoint Similarity)를 계산
# OKS = exp(-1.0 * (di ** 2) / (2 * alpha * base ** 2))
def compute_oks(v1, v2, alpha):
    scaling_factor, base = get_base(v1, v2)
    nose_to_hip_len = alpha * base

    oks = 0
    for i in range(len(v1)):
        # 학생(v2)의 좌표값에 대해 scaling_factor를 곱해 스승에 맞춤 
        v2_scaled = tuple(scaling_factor * val for val in v2[i])

        pointwise_dist = distance.euclidean(v1[i], v2_scaled)
        oks += math.exp(-1.0 * pointwise_dist ** 2 / (2 * nose_to_hip_len ** 2))

    result = oks / len(v1)
    return result

# PDJ (Percentage of Detected Joints)를 계산
def compute_pdj(v1, v2, alpha):
    scaling_factor, base = get_base(v1, v2)
    nose_to_hip_len = alpha * base

    pdj = 0
    for i in range(len(v1)):
        # 학생(v2)의 좌표값에 대해 scaling_factor를 곱해 스승에 맞춤 
        v2_scaled = tuple(scaling_factor * val for val in v2[i])
        #print(v2_scaled)
        pointwise_dist = distance.euclidean(v1[i], v2_scaled)
        if pointwise_dist < nose_to_hip_len:
             pdj += 1

    result = pdj * 1.0 / len(v1)
    return result

output_path_1 = '/content/output/json/yogaFriend_demo/'
output_path_2 = '/content/output/json/yogaFriend_demo_std/'

case1 = read_vec(output_path_1)
#print(len(case1))

case2 = read_vec(output_path_2)
#print(len(case2))

#alpha = 0.05
#alpha = 0.08
alpha = 0.1
#alpha = 0.02

print("simil_oks_1:", compute_oks(case1, case2, alpha))
print("simil_pdj_1:", compute_pdj(case1, case2, alpha))

Writing compute_pdj.py


In [15]:
!python compute_pdj.py > compute_pdj.txt

In [16]:
download_file()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>